# Data Import and Validation

This notebook covers importing e-commerce event data into SQL Server and validating the imported tables.


In [ ]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Try all available connection strings in order until one works
candidates = [
    # Named Pipes (if enabled)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=np:\\localhost\pipe\MSSQL$SQLEXPRESS\sql\query;DATABASE=Funnel_Purchases;Trusted_Connection=yes;",
    # LocalDB (if database is on LocalDB)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=(localdb)\\MSSQLLocalDB;DATABASE=Funnel_Purchases;Trusted_Connection=yes;",
    # SQL Auth (replace with your credentials if you have them)
    r"DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost\\SQLEXPRESS;DATABASE=Funnel_Purchases;UID=sa;PWD=YourPassword;",
]

query = "SELECT TOP 10 * FROM dbo.[2019-Oct]"

for conn_str in candidates:
    try:
        print(f"Trying: {conn_str}")
        conn = pyodbc.connect(conn_str, timeout=5)
        df = pd.read_sql(query, conn)
        print("SUCCESS! Data preview:")
        print(df.head())
        conn.close()
        break
    except Exception as e:
        print(f"FAILED: {type(e)} {e}")
        continue
else:
    print("All connection attempts failed. Please check your SQL Server configuration and credentials.")

In [ ]:
# --- Helper: get a working DB connection from candidates ---
def get_conn(timeout=30):
    """Try connection strings defined in `candidates` and return an open connection."""
    for conn_str in candidates:
        try:
            print(f"Trying connection: {conn_str}")
            conn = pyodbc.connect(conn_str, timeout=timeout)
            print('Connection established')
            return conn
        except Exception as e:
            print(f"Connection failed: {type(e).__name__}: {e}")
            continue
    raise RuntimeError('All connection attempts failed - please check SQL Server access and credentials')

# Example usage: conn = get_conn(); df = pd.read_sql_query(SQL, conn); conn.close()

## Optimization Note

- Only import necessary columns for validation.
- Use chunked loading for large tables.
- Example: `SELECT event_time, event_type FROM dbo.[2019-Oct]`
